# Travel Agent Planner with AgentNeo Integration

This Jupyter notebook demonstrates the integration of AgentNeo, a powerful tracing and monitoring tool, with a Travel Agent Planner system. AgentNeo provides seamless tracing capabilities for both function calls and AI model interactions, allowing for comprehensive analysis and debugging of complex systems.


# Setup and Imports
First, let's import the necessary libraries and set up our environment.

In [1]:
import os
import requests
from dotenv import load_dotenv
from litellm import completion
import openai
from openai import OpenAI

In [2]:
# Load environment variables
load_dotenv("enter your .env file path ")


# Initialize OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY")



In [3]:
# Initialize AgentNeo Package
import os
os.chdir('..')

from agentneo import AgentNeo, Tracer, Evaluation,launch_dashboard
# Initialize AgentNeo session
# Create project
neo_session = AgentNeo(session_name="test")

project_name = "ai_travel_agent_demo1"

try:
    neo_session.create_project(project_name=project_name)
    print("Project created successfully")
except:
    neo_session.connect_project(project_name=project_name)
    print("Project connected successfully")
# Start tracing
tracer = Tracer(session=neo_session)
tracer.start()

Project created successfully
Project 'ai_travel_agent_demo1' found.
Tracing Started.


# Travel Agent Tools
Now, let's define our Travel Agent Planner Tools with AgentNeo integration.

In [4]:
@tracer.trace_llm(name="llm_call")
def llm_call(prompt, max_tokens=512, model="gpt-3.5-turbo"):
    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=0.7,
    )

    return response.choices[0].message.content.strip()


# Tools outside agents
@tracer.trace_tool(name="weather_tool")
def weather_tool(destination):

    api_key = os.environ.get("OPENWEATHERMAP_API_KEY")
    base_url = "http://api.openweathermap.org/data/2.5/weather"

    params = {"q": destination, "appid": api_key, "units": "metric"}

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        weather_description = data["weather"][0]["description"]
        temperature = data["main"]["temp"]

        return f"{weather_description.capitalize()}, {temperature:.1f}°C"
    except requests.RequestException:
        return "Weather data not available."


@tracer.trace_tool(name="currency_converter_tool")
def currency_converter_tool(amount, from_currency, to_currency):
    api_key = os.environ.get("EXCHANGERATE_API_KEY")
    base_url = f"https://v6.exchangerate-api.com/v6/{api_key}/pair/{from_currency}/{to_currency}"

    try:
        response = requests.get(base_url)
        response.raise_for_status()
        data = response.json()

        if data["result"] == "success":
            rate = data["conversion_rate"]
            return amount * rate
        else:
            return None
    except requests.RequestException:
        return None


@tracer.trace_tool(name="flight_price_estimator_tool")
def flight_price_estimator_tool(origin, destination):
    # This is a mock function. In a real scenario, you'd integrate with a flight API.
    api_key = os.environ.get("FLIGHT_API_KEY")
    # Implement actual API call here
    return f"Estimated price from {origin} to {destination}: $500-$1000"


# Agent with persona
@tracer.trace_agent(name="itinerary_agent")
class ItineraryAgent:
    def __init__(self, persona="Itinerary Agent"):
        self.persona = persona

    def plan_itinerary(self, user_preferences, duration=3):
        itinerary_prompt = f"""
You are a travel expert named {self.persona}.
Based on the following user preferences, create a {duration}-day travel itinerary.

User Preferences:
{user_preferences}

Itinerary:
"""
        return llm_call(itinerary_prompt, max_tokens=512)





# Running the Analysis
Now let's create a  main function for our Travel Agent Planner which recall all tools 

In [5]:
# Main function

@tracer.trace_agent(name="travel_agent")
def travel_agent():
    print("Welcome to the Personalized Travel Planner!\n")

    # Get user input
    # user_input = input("Please describe your ideal vacation: ")
    user_input = "karela, 10 days, $100, nature"

    # Extract preferences
    preferences_prompt = f"""
Extract key travel preferences from the following user input:
"{user_input}"

Please provide the extracted information in this format:
Destination:
Activities:
Budget:
Duration (in days):
"""
    extracted_preferences = llm_call(preferences_prompt)
    print("\nExtracted Preferences:")
    print(extracted_preferences)

    # Parse extracted preferences
    preferences = {}
    for line in extracted_preferences.split("\n"):
        if ":" in line:
            key, value = line.split(":", 1)
            preferences[key.strip()] = value.strip()

    # Validate extracted preferences
    required_keys = ["Destination", "Activities", "Budget", "Duration (in days)"]
    if not all(key in preferences for key in required_keys):
        print("\nCould not extract all required preferences. Please try again.")
        return

    # Fetch additional information
    weather = weather_tool(preferences["Destination"])
    print(f"\nWeather in {preferences['Destination']}: {weather}")

    # origin = input("Please enter your departure city: ")
    origin = "delhi"
    flight_price = flight_price_estimator_tool(origin, preferences["Destination"])
    print(flight_price)

    # Plan itinerary
    itinerary_agent = ItineraryAgent()
    itinerary = itinerary_agent.plan_itinerary(
        extracted_preferences, int(preferences["Duration (in days)"])
    )
    print("\nPlanned Itinerary:")
    print(itinerary)

    # Currency conversion
    budget_amount = float(preferences["Budget"].replace("$", "").replace(",", ""))
    converted_budget = currency_converter_tool(budget_amount, "USD", "INR")
    if converted_budget:
        print(f"\nBudget in INR: {converted_budget:.2f} INR")
    else:
        print("\nCurrency conversion not available.")

    # Generate travel summary
    summary_prompt = f"""
Summarize the following travel plan:

Destination: {preferences['Destination']}
Activities: {preferences['Activities']}
Budget: {preferences['Budget']}
Duration: {preferences['Duration (in days)']} days
Itinerary: {itinerary}
Weather: {weather}
Flight Price: {flight_price}

Travel Summary:
"""
    travel_summary = llm_call(summary_prompt, max_tokens=2048)
    print("\nTravel Summary:")
    print(travel_summary)

In [6]:
# Main function to run the travel agent
def main():
    travel_agent()

# Ensure the script runs only when executed directly
if __name__ == "__main__":
    main()
    tracer.stop()


Welcome to the Personalized Travel Planner!



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Extracted Preferences:
Destination: karela
Activities: nature
Budget: $100
Duration (in days): 10

Weather in karela: Weather data not available.
Estimated price from goa to karela: $500-$1000


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Planned Itinerary:
Day 1-2: Arrival in Karela
- Check into a budget-friendly accommodation
- Explore the natural beauty of Karela, including its lush forests and scenic views
- Visit local parks and hiking trails for a nature-filled day

Day 3-4: Nature Excursions
- Take a guided nature walk through the Karela National Park
- Explore the nearby waterfalls and rivers for a refreshing experience
- Enjoy a picnic in the great outdoors

Day 5-6: Cultural Discovery
- Visit local villages and learn about the traditional way of life in Karela
- Attend a cultural performance showcasing traditional music and dances
- Sample local cuisine and delicacies

Day 7-8: Adventure Activities
- Try your hand at zip-lining through the forest canopy
- Go on a thrilling river rafting adventure
- Explore caves and gorges in the area

Day 9-10: Relaxation and Reflection
- Spend your last days in Karela unwinding at a spa or wellness retreat
- Reflect on your nature-filled adventures and memories made during 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
DEBUG:agentneo.tracing.agent_tracer:Successfully updated and committed AgentCallModel with id 2
DEBUG:agentneo.tracing.agent_tracer:Successfully updated and committed AgentCallModel with id 1



Travel Summary:
Destination: Karela
Activities: Nature exploration, cultural discovery, adventure activities, relaxation
Budget: $100
Duration: 10 days
Itinerary: Arrival and exploration of Karela's natural beauty, guided nature walks, cultural experiences, adventure activities like zip-lining and river rafting, relaxation at a spa or wellness retreat
Flight Price: Estimated at $500-$1000 from Goa to Karela

Overall, the travel plan offers a diverse range of activities and experiences in Karela within the budget of $100 for a 10-day trip.
Tracing Completed.
Data saved to the database and JSON file.



# Metrics Evaluation
Supported Metrics
Goal Decomposition Efficiency (goal_decomposition_efficiency)
Goal Fulfillment Rate (goal_fulfillment_rate)
Tool Call Correctness Rate (tool_call_correctness_rate)
Tool Call Success Rate (tool_call_success_rate)

In [7]:
# exe = Evaluation(session=neo_session, trace_id=tracer.trace_id)

In [8]:
# # run a single metric
# exe.evaluate(metric_list=['goal_decomposition_efficiency', 
#                          'goal_fulfillment_rate', 
#                          'tool_call_correctness_rate', 
#                          'tool_call_success_rate'])

In [9]:
# #print metric result
# metric_results = exe.get_results()
# metric_results

In [10]:
neo_session.launch_dashboard()

INFO:root:Port 3000 is busy. Finding an available port...
INFO:root:Using port 3028
INFO:root:Dashboard process started successfully
INFO:root:Dashboard launched successfully. Access it at: http://localhost:3028


### get the trace data

In [11]:
import requests
import json

trace_id = tracer.trace_id 
port = 2020 # port no of the falsk server
response = requests.get(f"http://localhost:{port}/api/analysis_traces/{trace_id}")

if response.status_code == 200:
    trace_data = response.json()
    print(json.dumps(trace_data, indent=2))
else:
    print(f"Error: {response.status_code}")
    print(response.text)

ConnectionError: HTTPConnectionPool(host='localhost', port=2020): Max retries exceeded with url: /api/analysis_traces/1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11a9bb4d0>: Failed to establish a new connection: [Errno 61] Connection refused'))